<img src="../img/SQLite-Python.jpg" alt="SQlite3 python" />

SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle.

The sqlite3 module was written by Gerhard Häring. It provides an SQL interface compliant with the DB-API 2.0 specification described by PEP 249, and requires SQLite 3.7.15 or newer.

<img src="../img/python-sqlite3-module-working.webp" alt="sqlite3 architecture"/>

# CREATE tables:

**Summary: in this tutorial, we will show you how to create tables in the SQLite database from the Python program using the sqlite3 module.**

To create a new table in an SQLite database from a Python program, you use the following steps:

1. First, create a Connection object using the connect() function of the sqlite3 module.
2. Second, create a Cursor object by calling the cursor() method of the Connection object.
3. Third, pass the CREATE TABLE statement to the execute() method of the Cursor object and execute this method.

For the demonstration, we will create two tables: projects and tasks as shown in the following database diagram:

<img src="../img/Python-SQLite-Sample-Tables.png" alt="schema diagram">

In [1]:
import sqlite3
from sqlite3 import Error

Let’s see how to create new tables in Python.

First, create a function called create_connection() that returns a Connection object which represents an SQLite database specified by the database file parameter db_file.

In [2]:
def create_connection(db_file):
    """ 
    create a database connection to the SQLite database specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

Second, create a function named create_table() that accepts a Connection object and an SQL statement. Inside the function, we call the execute() method of the Cursor object to execute the CREATE TABLE statement.

In [3]:
def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)

Third, create a main() function to create the  projects and tasks tables.

In [4]:
def main():
    database = r"pythonsqlite.db"

    sql_create_projects_table = """ 
        CREATE TABLE IF NOT EXISTS projects (
            id integer PRIMARY KEY,
            name text NOT NULL,
            begin_date text,
            end_date text
        ); 
    """

    sql_create_tasks_table = """
        CREATE TABLE IF NOT EXISTS tasks (
            id integer PRIMARY KEY,
            name text NOT NULL,
            priority integer,
            status_id integer NOT NULL,
            project_id integer NOT NULL,
            begin_date text NOT NULL,
            end_date text NOT NULL,
            FOREIGN KEY (project_id) REFERENCES projects (id)
        );
    """

    # create a database connection
    conn = create_connection(database)

    # create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_projects_table)

        # create tasks table
        create_table(conn, sql_create_tasks_table)
    else:
        print("Error! cannot create the database connection.")


if __name__ == '__main__':
    main()

Let’s verify if the program has created those tables successfully in the pythonsqlite.db database.

# Insert values in table:

**Summary: in this tutorial, you will learn how to insert rows into a table in the SQLite database from a Python program using the sqlite3 module.**

To insert rows into a table in SQLite database, you use the following steps:

1. First, connect to the SQLite database by creating a Connection object.
2. Second, create a Cursor object by calling the cursor method of the Connection object.
3. Third, execute an INSERT statement. If you want to pass arguments to the INSERT statement, you use the question mark (?) as the placeholder for each argument.

Let’s insert a new project into the projects table and some tasks into the tasks table that we created in the creating tables from a Python program tutorial.

<img src="../img/Python-SQLite-Sample-Database.jpg" alt="">

Create a function to insert a new project into the projects table.

In [5]:
def create_project(conn, project):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = '''
        INSERT INTO projects(name,begin_date,end_date)
        VALUES(?,?,?)
    '''
    cur = conn.cursor()
    cur.execute(sql, project)
    conn.commit()
    return cur.lastrowid

In above function, we used the  lastrowid attribute of the Cursor object to get back the generated id.

Then, create another function for inserting rows into the tasks table.

In [6]:
def create_task(conn, task):
    """
    Create a new task
    :param conn:
    :param task:
    :return:
    """

    sql = '''
        INSERT INTO tasks(name,priority,status_id,project_id,begin_date,end_date)
        VALUES(?,?,?,?,?,?)
    '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()
    return cur.lastrowid

After that, create the main() function that creates a new project and two tasks associated with the project.

In [7]:
def main():
    database = r"pythonsqlite.db"

    # create a database connection
    conn = create_connection(database)
    with conn:
        # create a new project
        project = ('Cool App with SQLite & Python', '2015-01-01', '2015-01-30');
        project_id = create_project(conn, project)

        # tasks
        task_1 = ('Analyze the requirements of the app', 1, 1, project_id, '2015-01-01', '2015-01-02')
        task_2 = ('Confirm with user about the top requirements', 1, 1, project_id, '2015-01-03', '2015-01-05')

        # create tasks
        create_task(conn, task_1)
        create_task(conn, task_2)


if __name__ == '__main__':
    main()

# Update values of table: 

**Summary: in this tutorial, we will show you how to update data in the SQLite database from a Python program using the sqlite3 module.**

To update data in a table from a Python program, you follow these steps:

1. First, create a database connection to the SQLite database using the connect() function. Once the database connection created, you can access the database using the Connection object.
2. Second, create a Cursor object by calling the cursor() method of the Connection object.
3. Third, execute the UPDATE statement by calling the execute() method of the Cursor object.

In this example we will update the priority, begin date, and end date of a specific task in the tasks table.

This update_task() function update a specific task:

In [8]:
def update_task(conn, task):
    """
    update priority, begin_date, and end date of a task
    :param conn:
    :param task:
    :return: project id
    """
    sql = ''' 
        UPDATE tasks
        SET 
            priority = ? ,
            begin_date = ? ,
            end_date = ?
        WHERE id = ?
    '''
    cur = conn.cursor()
    cur.execute(sql, task)
    conn.commit()

The following main() function creates a connection to the database located in current folder  and call the update_task() function to update a task with id 2:

In [9]:
def main():
    database = r"pythonsqlite.db"

    # create a database connection
    conn = create_connection(database)
    with conn:
        update_task(conn, (2, '2015-01-04', '2015-01-06', 2))


if __name__ == '__main__':
    main()

# SELECT records from table

**Summary: in this tutorial, we will show you step by step how to query data in SQLite from Python.**

To query data in an SQLite database from Python, you use these steps:

1. First, establish a connection to the SQLite database by creating a Connection object.
2. Next, create a Cursor object using the cursor method of the Connection object.
3. Then, execute a  SELECT statement.
4. After that, call the fetchall() method of the cursor object to fetch the data.
5. Finally, loop the cursor and process each row individually.

In the following example, we will use the tasks table created in the creating tables tutorial.

<img src="../img/Python-SQLite-Sample-Tables.png" alt = "table schema"/>

This function selects all rows from the tasks table and displays the data:

In [10]:
def select_all_tasks(conn):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM tasks")

    rows = cur.fetchall()

    for row in rows:
        print(row)

In the select_all_tasks() function, we created a cursor, executed the SELECT statement, and called the  fetchall() to fetch all tasks from the tasks table.

This function query tasks by priority:

In [11]:
def select_task_by_priority(conn, priority):
    """
    Query tasks by priority
    :param conn: the Connection object
    :param priority:
    :return:
    """
    cur = conn.cursor()
    cur.execute("SELECT * FROM tasks WHERE priority=?", (priority,))

    rows = cur.fetchall()

    for row in rows:
        print(row)

In the select_task_by_priority() function, we selected the tasks based on a particular priority. The question mark ( ?) in the query is the placeholder. When the cursor executed the SELECT statement, it substituted the question mark ( ?) by the priority argument. The  fetchall() method fetched all matching tasks by the priority.


This main() function creates a connection to the database  pythonsqlite.db and calls the functions to query all rows from the tasks table and select tasks with priority 1:

In [12]:
def main():
    database = r"pythonsqlite.db"

    # create a database connection
    conn = create_connection(database)
    with conn:
        print("1. Query task by priority:")
        select_task_by_priority(conn, 1)

        print("2. Query all tasks")
        select_all_tasks(conn)
        
if __name__ == '__main__':
    main()

1. Query task by priority:
(1, 'Analyze the requirements of the app', 1, 1, 1, '2015-01-01', '2015-01-02')
2. Query all tasks
(1, 'Analyze the requirements of the app', 1, 1, 1, '2015-01-01', '2015-01-02')
(2, 'Confirm with user about the top requirements', 2, 1, 1, '2015-01-04', '2015-01-06')


# Delete record from table

**Summary: this tutorial shows you how to delete data in the SQLite database from a Python program using the sqlite3 module.**

In order to delete data in the SQLite database from a Python program, you use the following steps:

1. First, establish a connection the SQLite database by creating a Connection object using the connect() function.
2. Second, to execute a DELETE statement, you need to create a Cursor object using the cursor() method of the Connection object.
3. Third, execute the  DELETE statement using the execute() method of the Cursor object. In case you want to pass the arguments to the statement, you use a question mark ( ?) for each argument.

The following delete_task() function deletes a task in the tasks table by id.

In [13]:
def delete_task(conn, id):
    """
    Delete a task by task id
    :param conn:  Connection to the SQLite database
    :param id: id of the task
    :return:
    """
    sql = 'DELETE FROM tasks WHERE id=?'
    cur = conn.cursor()
    cur.execute(sql, (id,))
    conn.commit()

The following delete_all_tasks() function deletes all rows in the tasks table.

In [14]:
def delete_all_tasks(conn):
    """
    Delete all rows in the tasks table
    :param conn: Connection to the SQLite database
    :return:
    """
    sql = 'DELETE FROM tasks'
    cur = conn.cursor()
    cur.execute(sql)
    conn.commit()

This main() function calls the create_connection() function and the delete_task() function to delete the task with id 2 from the tasks table:

In [15]:
def main():
    database = r"pythonsqlite.db"

    # create a database connection
    conn = create_connection(database)
    with conn:
        delete_task(conn, 2);
        # delete_all_tasks(conn);


if __name__ == '__main__':
    main()